# Solidity Jump Diffusion Logic

In [1]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize_scalar   
N = norm.cdf

def factorial(k):
    k_fact = np.math.factorial(k)
    return k_fact

def D1(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + (sigma**2)/2)*T) / (sigma*np.sqrt(T))
    return d1

def D2(d1, sigma, T):
    d2 = d1 - sigma * np.sqrt(T)
    return d2
    
def BS_CALL(S, K, T, r, sigma):
    d1 = D1(S, K, T, r, sigma)
    d2 = D2(d1, sigma, T)
    C = S * N(d1) - K * np.exp(-r*T)* N(d2)
    return C

def BS_PUT(S, K, T, r, sigma):
    d1 = D1(S, K, T, r, sigma)
    d2 = D2(d1, sigma, T)
    C = K*np.exp(-r*T)*N(-d2) - S*N(-d1)    
    return C

def delta_BS_CALL(S, K, T, r, sigma):
    d1 = D1(S,K,r,sigma,T)
    delta = N(d1)
    return delta

def delta_BS_PUT(S, K, T, r, sigma):
    d1 = D1(S,K,r,sigma,T)
    delta = N(-d1)
    return delta - 1

def RK(r, lam, m, k, T):
    r_k = r - lam*(m-1) + (k*np.log(m) ) / T
    return r_k
    
def SIGMA_K(sigma, k, v, T):
    sigma_k = np.sqrt( sigma**2 + (k* v** 2) / T)
    return sigma_k

def MJCnum(m, lam, T, k, k_fact):
    MJCnum = (np.exp(-m*lam*T) * (m*lam*T)**k / (k_fact))
    return MJCnum

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

def MERTON_CALL(S, K, T, r, sigma, m , v, lam):
    C = 0
    d1 = 0
    for k in range(5):
        r_k = RK(r, lam, m, k, T)
        sigma_k = SIGMA_K(sigma, k, v, T)
        k_fact = factorial(k)

        C += MJCnum(m, lam, T, k, k_fact) * BS_CALL(S, K, T, r_k, sigma_k)
        d1 += MJCnum(m, lam, T, k, k_fact) * D1(S, K, T, r, sigma) #delta_BS_CALL(S, K, T, r, sigma) 

    delta = N(d1)
    return (C, delta)


def MERTON_PUT(S, K, T, r, sigma, m , v, lam):
    C = 0
    d1 = 0
    for k in range(5):
        r_k = RK(r, lam, m, k, T)
        sigma_k = SIGMA_K(sigma, k, v, T)
        k_fact = factorial(k)
        
        C += MJCnum(m, lam, T, k, k_fact) * BS_PUT(S, K, T, r_k, sigma_k)
        d1 += MJCnum(m, lam, T, k, k_fact) * D1(S, K, T, r, sigma)
    delta = N(d1) - 1            
    return (C, delta)

# Python Jump Diffusion Logic

In [2]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize_scalar   
N = norm.cdf

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

def BS_PUT(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)    
    
def merton_jump_call(S, K, T, r, sigma, m , v, lam):
    p = 0
    for k in range(5):
        r_k = r - lam*(m-1) + (k*np.log(m) ) / T
        sigma_k = np.sqrt( sigma**2 + (k* v** 2) / T)
        k_fact = np.math.factorial(k)
        p += (np.exp(-m*lam*T) * (m*lam*T)**k / (k_fact))  * BS_CALL(S, K, T, r_k, sigma_k)
    
    return p 

def merton_jump_put(S, K, T, r, sigma, m , v, lam):
    p = 0 # price of option
    for k in range(5):
        r_k = r - lam*(m-1) + (k*np.log(m) ) / T
        sigma_k = np.sqrt( sigma**2 + (k* v** 2) / T)
        k_fact = np.math.factorial(k) # 
        p += (np.exp(-m*lam*T) * (m*lam*T)**k / (k_fact)) \
                    * BS_PUT(S, K, T, r_k, sigma_k)
    return p 

In [3]:
S = 100
K = 100
T = 1
r = 1
m = 1 # meean of jump size
v = 1 # standard deviation of jump
lam = 1 # intensity of jump i.e. number of jumps per annum
sigma = 1 # annaul standard deviation , for weiner process

call = MERTON_CALL(S, K, T, r, sigma, m , v, lam)
put = MERTON_PUT(S, K, T, r, sigma, m , v, lam)

call1 = merton_jump_call(S, K, T, r, sigma, m , v, lam)
put1 = merton_jump_put(S, K, T, r, sigma, m , v, lam)

In [4]:
print("solidity logic:", call)
#print("solidity logic:", put)

print("python logic:", call1)
#print("python logic:", put1)

solidity logic: (72.8366665717326, 0.932478844920132)
python logic: 72.8366665717326
